### App to find rare illnesses in ADN datasets using mongodb and apache spark

In [11]:
%pip install pymongo
%pip install pyspark
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 350.1 kB/s eta 0:00:000:00:01m eta 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 141.9 kB/s eta 0:00:00m eta 0:00:020:00:04
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from pymongo import MongoClient

In [22]:
client = MongoClient("mongodb://localhost:27017/")

adn_db = client["adn_db"]
illness_db = client["illness_db"]

adn_col = adn_db["sequences"]
illness_col = illness_db["rare_illnesses"]

adn_exists = adn_col.estimated_document_count() > 0
illness_exists = illness_col.estimated_document_count() > 0

if not adn_exists:
    print("Inserting ADN data...")
    adn_data = [
        {"sample_id": "S001", "sequence": "ATGCGTACGTAGCTAG", "source": "Patient A"},
        {"sample_id": "S002", "sequence": "CGTAGCTAGCTAGCTA", "source": "Patient B"},
    ]
    adn_col.insert_many(adn_data)
    print("ADN data inserted.")
else:
    print("ADN data already exists.")

if not illness_exists:
    print("Inserting illness data...")
    illness_data = [
        {
            "name": "Cystic Fibrosis",
            "symptoms": "Persistent cough, lung infections, poor growth",
            "related_genes": "CFTR",
        },
        {
            "name": "Huntington's Disease",
            "symptoms": "Movement issues, cognitive decline, psychiatric problems",
            "related_genes": "HTT",
        },
        {
            "name": "Tay-Sachs Disease",
            "symptoms": "Muscle weakness, seizures, vision and hearing loss",
            "related_genes": "HEXA",
        },
    ]
    df = pd.DataFrame(illness_data)
    df = df.head(10).rename(columns={"State": "name"})
    df["symptoms"] = "Symptom 1, Symptom 2"
    df["related_genes"] = "GENE1, GENE2"
    illness_data = df[["name", "symptoms", "related_genes"]].to_dict(orient="records")
    illness_col.insert_many(illness_data)
    print("Illness data inserted.")
else:
    print("Illness data already exists.")


ADN data already exists.
Illness data already exists.


In [25]:
for doc in adn_col.find():
    print(doc)

df_adn = pd.DataFrame(list(adn_col.find()))
display(df_adn)

df_illness = pd.DataFrame(list(illness_col.find()))
display(df_illness)


{'_id': ObjectId('67fda572c260f93aa1da86cf'), 'sample_id': 'S001', 'sequence': 'ATGCGTACGTAGCTAG', 'source': 'Patient A'}
{'_id': ObjectId('67fda572c260f93aa1da86d0'), 'sample_id': 'S002', 'sequence': 'CGTAGCTAGCTAGCTA', 'source': 'Patient B'}


,_id,sample_id,sequence,source
0,67fda572c260f93aa1da86cf,S001,ATGCGTACGTAGCTAG,Patient A
1,67fda572c260f93aa1da86d0,S002,CGTAGCTAGCTAGCTA,Patient B


,_id,name,symptoms,related_genes
0,67fda63fc260f93aa1da86d6,Cystic Fibrosis,"Symptom 1, Symptom 2","GENE1, GENE2"
1,67fda63fc260f93aa1da86d7,Huntington's Disease,"Symptom 1, Symptom 2","GENE1, GENE2"
2,67fda63fc260f93aa1da86d8,Tay-Sachs Disease,"Symptom 1, Symptom 2","GENE1, GENE2"
